# StationSim parametric study
    authors: A. West
    created: 2019-06-24
    version: 0.2 (jupyter)
Here we'll show how to change parameters.  And try a little profiling.

## Import
_For more information about this part read the basic experiment._  

In [1]:
from sys import path
path.append('..')
from models.stationsim import Model
from time import strftime, time
id = strftime('%y%m%d_%H%M%S')
from numpy.random import seed
seed(1)

`dict_to_csv()` converts a two layer dictionary into comma seperated string.  Useful for analysing several variables.  `roundp()` is a significant figure rounding method, use `sig_fig` for you accuracy.  
_There is no need to understand this method._

In [2]:
def dict_to_csv(mydict, sig_fig=16):
    roundp = lambda x,p: float(f'%.{p-1}e'%x)
    csv_str, lines = '', []
    for i,row in enumerate(mydict):
        if i==0:
            header = ', '.join(k for k,_ in mydict[row].items()) + ',\n'
        line = ', '.join(f'{roundp(v, sig_fig)}' for _,v in mydict[row].items()) + f', {row}'
        lines.append(line)
    csv_str = header + '\n'.join(lines)
    return csv_str

## Pop,Sep Study
Since the main scale for runtime is population.  By increasing separation we can mitigate the interaction loss with a population reduction.  As if studying a smaller corridor.

In [3]:
analytics = {}
for pop, sep in [(100, 5), (300, 3), (700, 2)]:
    t = time()
    model = Model(pop_total=pop, separation=sep, do_print=False)
    for _ in range(model.step_limit):
        model.step()
    analytics[str(model.params_changed)] = {'Compute Time': time()-t, **model.get_analytics()}

### Results

In [8]:
print(dict_to_csv(analytics, 4))
# print(csv_str, file=open(f'{id}_pop_sep_study.csv', 'w'))

Compute Time, Finish Time, Total, Active, Finished, Time Taken, Time Expected, Time Delay, Collisions, Wiggles,
3.124, 1913.0, 100.0, 0.0, 100.0, 579.6, 419.6, 160.0, 790.9, 54.28, {}
9.521, 2101.0, 300.0, 0.0, 300.0, 588.5, 433.0, 155.5, 699.4, 67.66, {'pop_total': 300, 'separation': 3}
24.79, 2161.0, 700.0, 0.0, 700.0, 618.8, 446.3, 172.6, 704.6, 84.71, {'pop_total': 700, 'separation': 2}
5.878, 1987.0, 100.0, 0.0, 100.0, 653.2, 488.5, 164.7, 1945.0, 50.56, {'speed_steps': 9}
3.138, 1781.0, 100.0, 0.0, 100.0, 562.9, 457.8, 105.1, 718.4, 45.63, {'speed_steps': 5}
2.558, 2074.0, 100.0, 0.0, 100.0, 556.9, 437.6, 119.2, 406.1, 49.85, {'speed_steps': 2}
2.221, 2114.0, 100.0, 0.0, 100.0, 545.6, 434.6, 111.0, 201.0, 66.4, {'speed_steps': 1}


## Speeds Study
The default average number of speeds is set to 3.  This is the number of times the agent will slow down before wiggling.  Increasing the number will increase the number of interactions/collisions, but will decrease the number of wiggles.  Furthermore, it will increase compute time linearly.

In [5]:
# analytics = {}
for s in (9,5,2,1):
    t = time()
    model = Model(speed_steps=s, do_print=False)
    for _ in range(model.step_limit):
        model.step()
    analytics[str(model.params_changed)] = {'Compute Time': time()-t, **model.get_analytics()}

### Results
By not resetting the analytics dictionary we append the new tests.  And compare a larger parameter space.  
Although we'd expect more checks to increase runtime, it doesn't seem to add much on.

In [7]:
print(dict_to_csv(analytics, 4))
# print(csv_str, file=open(f'{id}_speeds_study.csv', 'w'))

Compute Time, Finish Time, Total, Active, Finished, Time Taken, Time Expected, Time Delay, Collisions, Wiggles,
3.124, 1913.0, 100.0, 0.0, 100.0, 579.6, 419.6, 160.0, 790.9, 54.28, {}
9.521, 2101.0, 300.0, 0.0, 300.0, 588.5, 433.0, 155.5, 699.4, 67.66, {'pop_total': 300, 'separation': 3}
24.79, 2161.0, 700.0, 0.0, 700.0, 618.8, 446.3, 172.6, 704.6, 84.71, {'pop_total': 700, 'separation': 2}
5.878, 1987.0, 100.0, 0.0, 100.0, 653.2, 488.5, 164.7, 1945.0, 50.56, {'speed_steps': 9}
3.138, 1781.0, 100.0, 0.0, 100.0, 562.9, 457.8, 105.1, 718.4, 45.63, {'speed_steps': 5}
2.558, 2074.0, 100.0, 0.0, 100.0, 556.9, 437.6, 119.2, 406.1, 49.85, {'speed_steps': 2}
2.221, 2114.0, 100.0, 0.0, 100.0, 545.6, 434.6, 111.0, 201.0, 66.4, {'speed_steps': 1}


## _Default Model_
_The default model is designed to run quickly and efficiently.  Hence setting a low population and high separation._

_For more information about the speed of this model, check the profiling experiments._